In [2]:
import torch 
import torch.nn.functional as F 
import matplotlib.pyplot as plt 
%matplotlib inline

words = open('/Users/mli/Documents/Projects/karpathy/makemore-master/names.txt', 'r').read().splitlines()

In [30]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [31]:
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print((''.join(itos[i] for i in context), '->', itos[ix]))
        context = context[1:] + [ix]        
X = torch.tensor(X)
Y = torch.tensor(Y)


emma
('...', '->', 'e')
('..e', '->', 'm')
('.em', '->', 'm')
('emm', '->', 'a')
('mma', '->', '.')
olivia
('...', '->', 'o')
('..o', '->', 'l')
('.ol', '->', 'i')
('oli', '->', 'v')
('liv', '->', 'i')
('ivi', '->', 'a')
('via', '->', '.')
ava
('...', '->', 'a')
('..a', '->', 'v')
('.av', '->', 'a')
('ava', '->', '.')
isabella
('...', '->', 'i')
('..i', '->', 's')
('.is', '->', 'a')
('isa', '->', 'b')
('sab', '->', 'e')
('abe', '->', 'l')
('bel', '->', 'l')
('ell', '->', 'a')
('lla', '->', '.')
sophia
('...', '->', 's')
('..s', '->', 'o')
('.so', '->', 'p')
('sop', '->', 'h')
('oph', '->', 'i')
('phi', '->', 'a')
('hia', '->', '.')


In [32]:
X.shape

torch.Size([32, 3])

In [33]:
# two dimensional embedding for 27 chars
# can think of this as weights in a linear layer of neural net
C = torch.randn((27,2))
C

tensor([[ 0.7063,  1.6463],
        [ 0.4621,  1.6692],
        [ 0.3124,  0.5896],
        [ 0.4606, -1.6767],
        [ 0.1185, -0.4073],
        [-0.7021, -0.5707],
        [ 0.1697,  1.7292],
        [-0.8571, -1.0705],
        [ 0.9167, -0.2094],
        [ 0.3646,  0.1167],
        [ 1.7574, -1.1307],
        [ 0.1896, -0.6268],
        [-0.0030,  0.5312],
        [ 0.7462, -0.0652],
        [-0.0160, -0.2930],
        [ 1.3795,  0.8247],
        [-0.1766, -1.2988],
        [-0.3645,  0.9477],
        [-0.2276, -2.2501],
        [-1.4131,  0.2363],
        [ 1.0209,  1.4918],
        [ 1.3886, -0.5729],
        [-0.9563, -0.4684],
        [ 0.2255, -1.3072],
        [ 0.2717,  0.3353],
        [ 0.0048,  0.3639],
        [-1.0808,  0.7474]])

In [34]:
emb = C[X] # embedding vector of every training example
emb.shape
# print(emb[1])
# print(X[1])
# print(emb)

torch.Size([32, 3, 2])

In [35]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [36]:
a = emb.view(-1,6)
a.shape # 32, 6
W1.shape # 6, 100
b = emb.view(-1,6) @ W1
b.shape # 32, 100
b


tensor([[ 3.0925,  1.0925,  0.1506,  ...,  0.1078, -0.7565,  0.0467],
        [ 2.3679,  1.8694,  4.0407,  ..., -1.3999, -0.5730,  2.1268],
        [-1.9116, -4.7800, -0.6553,  ..., -0.0220, -0.2503, -1.4585],
        ...,
        [-1.9944, -4.5693, -2.4963,  ...,  1.2379,  0.6951,  1.2480],
        [ 1.2197,  1.0177, -1.7889,  ...,  0.6351,  0.6296,  1.9883],
        [ 1.1820,  2.1155, -2.5621,  ...,  2.0717,  0.4074,  1.1375]])

In [37]:
# torch.cat((emb[:,0,:], emb[:,1,:], emb[:,2,:]), 1)
# torch.cat(torch.unbind(emb,1), 1)
emb.view([32,6]) # most efficient

h = torch.tanh(emb.view(-1,6) @ W1 + b1)
h.shape


torch.Size([32, 100])

In [38]:
emb.view(-1,6)
emb.shape
# h

torch.Size([32, 3, 2])

In [39]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)
logits = h @ W2 + b2    
logits.shape

torch.Size([32, 27])

In [40]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [41]:
loss = -prob[torch.arange(32), Y].log().mean()

In [42]:
block_size = 3
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)        
        context = context[1:] + [ix]        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [43]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
# counts = logits.exp()
# prob = counts / counts.sum(1, keepdim=True)
# loss = -prob[torch.arange(32), Y].log().mean()
loss = F.cross_entropy(logits,Y)
loss

tensor(19.5052, grad_fn=<NllLossBackward0>)

In [44]:
step = 0.01
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre

lri = []
lossi = []
for i in range(10000):
    # mini batch
    ix = torch.randint(0,X.shape[0], (256,))    

    # forward
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Y[ix])    
    print(loss.item())

    # backward
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    # step = lrs[i]    
    for p in parameters:
        p.data -= step * p.grad

    # track stats
    # lri.append(lre[i])
    # lossi.append(loss.item())

# print(loss.item())

19.973928451538086
19.13915252685547
18.658010482788086
18.54987144470215
19.47683334350586
17.739774703979492
17.82643699645996
17.913251876831055
16.94552230834961
17.12594223022461
17.082826614379883
16.960433959960938
15.711029052734375
17.24376106262207
16.634296417236328
17.479663848876953
16.638235092163086
16.276575088500977
15.801896095275879
16.02388572692871
16.06197738647461
15.437793731689453
15.875149726867676
15.874004364013672
16.021263122558594
16.018444061279297
15.215315818786621
15.951698303222656
14.86094856262207
15.0045166015625
13.83184814453125
15.209480285644531
15.067451477050781
14.061720848083496
14.414664268493652
14.031304359436035
15.44096851348877
14.236123085021973
14.06861400604248
14.654520988464355
14.41477108001709
13.158166885375977
12.987738609313965
14.97972583770752
14.272705078125
13.375251770019531
13.668818473815918
13.349946022033691
13.687116622924805
13.196416854858398
13.221224784851074
12.412603378295898
13.2615385055542
12.728001594543

In [45]:
# plt.plot(lri, lossi)

In [46]:
emb = C[X]
h = torch.tanh(emb.view(-1,6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits,Y)    
loss

tensor(2.5622, grad_fn=<NllLossBackward0>)

In [47]:
# training split, validation split, test split
# 80%, 10%, 10%

# tuning hyperparameters

In [49]:
# g = torch.Generator().manual_seed(2147483647)
for _ in range(20):
    string = ''
    context = [0] * block_size
    
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1,-1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        if ix == 0:
            break
        string += itos[ix]
    print(string)

maraiua
dsca
dvi
kh
acae
ralvi
edmi
shan
manayle
ielsln
daeneerarin
beca
d
leafa
tor
sri
pnron
shar
rhisi
mue


In [50]:
logits

tensor([[  2.5762,   0.9334,  -2.1115,  -3.0735,  -0.6315,   1.3606,  -7.3956,
          -6.9196,   0.4522,   0.3656,  -2.4173,  -1.7922,   0.5846,  -1.7092,
           2.0003,  -0.1859, -13.2152, -12.3328,  -0.8452,  -1.0588,  -0.0917,
          -1.2510,  -2.9831,  -3.1512,  -1.5856,   0.8095,  -3.9433]],
       grad_fn=<AddBackward0>)

In [51]:
logits.shape

torch.Size([1, 27])

In [151]:
probs.sum()

tensor(1., grad_fn=<SumBackward0>)

In [52]:
C

tensor([[ 0.3739, -0.3525],
        [ 0.1734, -0.3478],
        [-0.0299, -0.3252],
        [-0.0553, -0.4044],
        [-0.1509, -0.3519],
        [ 0.0780, -0.3141],
        [-0.1159, -0.3590],
        [ 0.5453,  2.3571],
        [-0.1806, -0.2373],
        [-0.0091, -0.3179],
        [-0.2640, -0.3325],
        [-0.1751, -0.3593],
        [ 0.3908,  2.1780],
        [-0.2101, -0.3419],
        [-0.1335, -0.1958],
        [ 0.1381, -0.3392],
        [ 0.3640,  1.9839],
        [-0.3224, -0.5357],
        [-0.1687, -0.3815],
        [-0.0052, -0.4281],
        [-0.1190, -0.4237],
        [ 0.5170,  2.2147],
        [-0.2648, -0.4113],
        [-0.1771, -0.4209],
        [-0.1796, -0.2379],
        [-0.0936, -0.2302],
        [-0.1756, -0.3748]], requires_grad=True)